In [466]:
import warnings
warnings.filterwarnings("ignore")


In [467]:
# Load libraries
import numpy as np
from numpy import arange
from matplotlib import pyplot
import pandas as pd
pd.options.display.max_colwidth = 500

In [468]:
df = pd.read_csv("incidents_upd.csv")

In [469]:
df.head(2)

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager name)\r\n-checked the user name in ad and reset the password.\r\n-advised the user to login and check.\r\n-caller confirmed that he was able to login.\r\n-issue resolved.,spxjnwir pjlcoqds,GRP_0
1,outlook,"\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail.com\r\n\r\nhello team,\r\n\r\nmy meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this?\r\n\r\nkind",hmjdrvpb komuaywn,GRP_0


In [470]:
df["Assignment group"].value_counts(normalize=True)
df= df.groupby('Assignment group').filter(lambda x : len(x)>0)

In [471]:
df["Assignment group"].nunique()

74

In [472]:
#df["Caller"].value_counts
df.info()
print("number of records:",df.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8500 entries, 0 to 8499
Data columns (total 4 columns):
Short description    8498 non-null object
Description          8499 non-null object
Caller               8500 non-null object
Assignment group     8500 non-null object
dtypes: object(4)
memory usage: 332.0+ KB
number of records: (8500, 4)


In [473]:
df[df["Description"].isnull()]
df.isnull().sum()

Short description    2
Description          1
Caller               0
Assignment group     0
dtype: int64

In [474]:
# Replace Nan values in Description to THE - since this stop word will be eliminated in the vectorization
df["Short description"].fillna(df["Description"], inplace=True)
df["Description"].fillna(df["Short description"], inplace=True)

In [475]:
df[df["Description"].isnull()]
df.isnull().sum()

Short description    0
Description          0
Caller               0
Assignment group     0
dtype: int64

In [476]:
df1 = df

In [477]:
df1["Short description"]= df1["Short description"].apply(lambda X: X.lower())
df1["Description"]= df1["Description"].apply(lambda X: X.lower())

In [478]:
import re
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [479]:
def remove_punct(text):
    text = re.sub('[0-9]+|\n|\r|[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', ' ', text)
    return text

In [480]:
df1["Short description"]= df1["Short description"].apply(lambda X: remove_punct(X))
df1["Description"]= df1["Description"].apply(lambda X: remove_punct(X))

In [481]:
df1.head(3)

,Short description,Description,Caller,Assignment group
0,login issue,verified user details employee manager name checked the user name in ad and reset the password advised the user to login and check caller confirmed that he was able to login issue resolved,spxjnwir pjlcoqds,GRP_0
1,outlook,received from hmjdrvpb komuaywn gmail com hello team my meetings skype meetings etc are not appearing in my outlook calendar can somebody please advise how to correct this kind,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,received from eylqgodm ybqkwiam gmail com hi i cannot log on to vpn best,eylqgodm ybqkwiam,GRP_0


In [482]:
short_keywords = dict()
long_keywords = dict()

In [483]:
def checkMultiWords(keyword, value):
    present = False
    if keyword in ['milano','datacenter']:
            print(word)
    for word in keyword.split('&&'):
        if word in value:
            present = True
        else: 
            present = False
            break
    return present

In [484]:
def getGroup(short, description):
    for group in short_keywords:
        for keyword in short_keywords[group]:
            if keyword in short:
                return 'GRP_' + str(group)
            else:
                if ('&&' in keyword) and (checkMultiWords(keyword, short)):
                    return 'GRP_' + str(group) 
                
    for group in long_keywords:
        for keyword in long_keywords[group]:
            if keyword in description:
                return 'GRP_' + str(group)
            else:
                if ('&&' in keyword) and (checkMultiWords(keyword, description)):
                    return 'GRP_' + str(group)

In [489]:
long_keywords[61] = ['calibration system']
short_keywords[64] = ['change in report']
short_keywords[35] = ['erp kp']
short_keywords[67] = ['unable && complete forecast']
long_keywords[70] = ['training && signature']
long_keywords[73] = ['unable && hr tool && oneteam']
long_keywords[54] = ['customer stock', 'batch && plant && goods']
long_keywords[57] = ['access && dob report']
long_keywords[69] = ['quarantine && european', 'milano && edml']
short_keywords[71] = ['production files']
long_keywords[71] = ['production feed']
short_keywords[38] = ['delete && charm', 'hpqc && user']
long_keywords[56] = ['source mac address && android','infoblox && usa datacenter', 'hiyhugins']
long_keywords[58] = ['engg && stopped && restart']
long_keywords[63] = ['android && install && crm','not receiving && e mail && phone', 'receive && call && iphone']
long_keywords[68] = ['my expense report && submit', 'expense report && infortype']
short_keywords[32] = ['wg  po']
long_keywords[32] = ['edi file && rejected']
short_keywords[66] = ['installing && cutview', 'update && cutview','tess application && run']
long_keywords[43] = ['shop floor app && pcd', 'shopfloor && production order', 'shop floor && production order',
                    'shop floor app && password', 'shop floor app && access']
long_keywords[46] = ['nx && programdnty', 'engineering drawing tool && kommt','customization && engineering tool',
                    'engineering tool && error log']
long_keywords[49] = ['prgramdntyme','erp && ecs tracking', 'login && search and analytics',
                    'cursor springt', 'missing records && nightly && milano']
long_keywords[59] = ['network problems && eu tool', 'transfer && eu tool && erp', 'password && eu tool','eu tool daten']
long_keywords[51] = ['credit component', 'road king && page', 'issue && all environments']

In [491]:
df1['target'] = df1.apply(lambda x: getGroup(x['Short description'], x['Description']), axis=1)
df1[~df1.target.isnull()]

,Short description,Description,Caller,Assignment group,target
238,erp pi and msd crm connectivity issue serirtce certificate renewal,hi all we have a connectivity issue between erp pi and msd crm due to certificate renewal on azure websites and that certificate is not yet updated on erp pi side issue changes to data records accounts sales area data partner functions contacts leads are not being processed from erp pi to worked with it msd crm and vice versa in all environments prod qa and dev issue start time th oct pm troubleshooting steps when tryhdty from customer master team reported the is...,kgytujhe bonhwzrx,GRP_14,GRP_51
401,need access to erp kp,need access to kp to enter forecast for it cost centers i had requested and received this access at the end of august access appears to be gone again need by,etvendormhd xpslzunb,GRP_35,GRP_35
461,hpqc delivers error message user is not maintained in the project fy release please contact hpqc admin,my user id thrydksd iâ´m involved in uat uacyltoe hxgayczeing for ec ecc and was asked to re install hpqc client installation kit what iâ´ve already done with below links when i start hpqc and enter my windows user name and windows password i receive following error message user is not maintained in the project fy release please contact hpqc admin,iavozegx jpcudyfi,GRP_38,GRP_38
626,production order number issue,received from zuxcfonv nyhpkrbe gmail com gso currently erp system production order number cannot be found after order number was identify enter in shopfloor everything is fine before erp routine maintenance please open a ticket for escalate the responsible team that s very urgent cid image jpg d de bb fa cid image png d dea d a best,zuxcfonv nyhpkrbe,GRP_43,GRP_43
841,error with customization engineering tool erp plam and company tools,hi team i need your help with two error the customization the engineering tool the two errors attachment in ticket erro computer lpawty diwhdd jwddkwor and lpawty gsotqxfi lidunfjg if you have more doubts please contatc me,qasdhyzm yuglsrwx,GRP_46,GRP_46
1143,please redirect to local it word document doc broken,das folgende word dokument kann nicht mehr bearbeitet werden \\edksm \qs\zeugnis kunden\ \ \ doc der cursor springt stã¤ndig auf seite,cwtrgeup nxctkmzo,GRP_49,GRP_49
1209,help with programdnty nx,hi we need your help with programdnty nx when the users tiaghry santhuy and pvimkcfw sarxkfvj click in gkad the nx close can we do make something to solve this problem,pvimkcfw sarxkfvj,GRP_46,GRP_46
1637,receiving product in logical warehouse plant batch is required,when using migo goods receipt to receive product into plant batch information is required to process it has begun to happen more frequently over the past month the following rqfhiong zkwfqagbs are examples as to where this has occurred mm and,qtrmxlgv dfruzvam,GRP_54,GRP_54
1757,please provide ip for our users,å‘ä»¶äºº white ben ben white hr tool com å‘é€æ—¶é—´ å¹´ æœˆ æ—¥ æ”¶ä»¶äºº eva li dyhtuiel hiyhugins company com stefyty parkeyhrt æš„é€ haiwei liang hr tool com patience rob ä¸»é¢˜ re company intepmov imjukbqhing plan hi eva can you work with your local it team to determine your desktop ip address and provide it to dyhtuiel and stefyty hi dyhtuiel stefyty eva is a miowvyrs qkspyrdm located in the apac office please check that the routing is in place from her...,krnzfoct mnjbuedp,GRP_56,GRP_56
1886,i cannot access the dob report i was able to access this before i need this report for the global busienss,i cannot access the dob report i was able to access this before i need this report for the global busienss,vsbtygin oufhtbas,GRP_57,GRP_57
